In [3]:
%reload_ext autoreload
%autoreload 2
import sys
import os
import importlib
script_dir = os.getcwd()  
sys.path.append(os.path.abspath(os.path.join(script_dir, '..')))


In [4]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")  # read in tabular-cleaned-data parquet file into a data frame
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,11.0,7.0,4.0,3.0,4.0,9.0,19.0,2022-01-29,4,17.0
1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,5.0,4.0,10.0,7.0,5.0,9.0,10.0,2022-01-30,4,9.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,3.0,...,8.0,7.0,8.0,5.0,5.0,10.0,0.0,2022-01-31,4,3.0
3,1.0,1.0,0.0,0.0,0.0,3.0,2.0,3.0,4.0,5.0,...,3.0,16.0,7.0,1.0,0.0,1.0,3.0,2022-02-01,4,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,2.0,...,3.0,8.0,3.0,0.0,4.0,4.0,3.0,2022-02-02,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-27,199,0.0
88290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-28,199,0.0
88291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-29,199,0.0
88292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-30,199,0.0


In [5]:
from datetime import datetime
from src.data_split import train_test_split # import func from py file

X_train, y_train, X_test, y_test = train_test_split(df, cutoff_date=datetime(2022, 6, 1, 0, 0, 0), target_column_name="target_rides_next_hour")

print(f"X_train: {X_train.shape}") # (examples-in-train, num-features)
print(f"y_train: {y_train.shape}") # (examples-in-train,) 1D beceause only one target
print(f"X_test: {X_test.shape}")   # (examples-in-test, num-features)
print(f"y_test: {y_test.shape}")   # (examples-in-test,) 1D beceause only one target
# examples-in-train != examples-in-test
# x_train
# y_train

X_train: (32226, 674)
y_train: (32226,)
X_test: (56068, 674)
y_test: (56068,)


In [11]:
import lightgbm as lgb

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
past_rides_columns = [c for c in X_train.columns if c.startswith("rides_")] # iterate through all columns in x-train-df and select all whose name starts with rides_
X_train_only_numeric = X_train[past_rides_columns]  # subsets of the x-train-data-frame by only select columns in past-rides-columns

In [ ]:
model = lgb.LGBMRegressor()                  # create lgb-model-regression-obj
model.fit(X_train_only_numeric, y_train)    # train model passing in x-train-with only rides in previous hour columns and y-train
# the features are only the previous rides, faster than xgboost

In [ ]:
X_test_only_numeric = X_test[past_rides_columns]  # filter x-test-df whose columns by only selecting columns in past-rides-column
predictions = model.predict(X_test_only_numeric)  # lgb-model predict passing in testing-data

from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)  # built-in-func takes in testing-labels and predictions-vector, outputs mean-absolute-error
print(f"MAE XG-BOOST: {test_mae=:.4f}")
